<h1 div class='alert alert-success'><center> TPS-Oct: ponto de partida (EDA, linha de base XGB)</center></h1>

![](https://storage.googleapis.com/kaggle-competitions/kaggle/26480/logos/header.png?t=2021-04-09-00-57-05)

# Descrição de dados

Para esta competição, você vai prever se um cliente fez uma reclamação sobre uma apólice de seguro. A verdade fundamental claimtem valor binário, mas uma previsão pode ser qualquer número de 0.0 para 1.0, representando a probabilidade de uma reclamação. Os recursos neste conjunto de dados foram tornados anônimos e podem conter valores ausentes.
arquivos

- `train.csv`: os dados de treinamento com o alvo claimcoluna
- `test.csv`: o conjunto de teste; você estará prevendo o claimpara cada linha neste arquivo
- `sample_submission.csv`:  um arquivo de envio de amostra no formato correto

# <div class="alert alert-success">  1. IMPORTAÇÕES </div> 


## 1.1. Bibliotecas 

In [ ]:
import warnings
import random
import os
import gc
#import cudf

In [ ]:
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt 
import seaborn           as sns
import joblib            as jb

In [ ]:
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing   import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, QuantileTransformer
from sklearn.impute          import SimpleImputer
from sklearn                 import metrics

In [ ]:
import xgboost               as xgb
import catboost              as ctb

## 1.2. Funções
Aqui centralizamos todas as funções desenvolvidas durante o projeto para melhor organização do código.

In [ ]:
def jupyter_setting():
    
    %matplotlib inline
      
    #os.environ["WANDB_SILENT"] = "true" 
    #plt.style.use('bmh') 
    #plt.rcParams['figure.figsize'] = [20,15]
    #plt.rcParams['font.size']      = 13
     
    pd.options.display.max_columns = None
    #pd.set_option('display.expand_frame_repr', False)

    warnings.filterwarnings(action='ignore')
    warnings.simplefilter('ignore')
    warnings.filterwarnings('ignore')
    #warnings.filterwarnings(category=UserWarning)

    warnings.filterwarnings('ignore', category=DeprecationWarning)
    warnings.filterwarnings('ignore', category=FutureWarning)
    warnings.filterwarnings('ignore', category=RuntimeWarning)
    warnings.filterwarnings('ignore', category=UserWarning)
    #warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

    pd.set_option('display.max_rows', 150)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.max_colwidth', None)

    icecream = ["#00008b", "#960018","#008b00", "#00468b", "#8b4500", "#582c00"]
    #sns.palplot(sns.color_palette(icecream))
    
    return icecream

icecream = jupyter_setting()

# Colors
dark_red = "#b20710"
black    = "#221f1f"
green    = "#009473"
myred    = '#CD5C5C'
myblue   = '#6495ED'
mygreen  = '#90EE90'

cols= [myred, myblue,mygreen]

In [ ]:
colors = ["lightcoral", "sandybrown", "darkorange", "mediumseagreen",
          "lightseagreen", "cornflowerblue", "mediumpurple", "palevioletred",
          "lightskyblue", "sandybrown", "yellowgreen", "indianred",
          "lightsteelblue", "mediumorchid", "deepskyblue"]

In [ ]:
def missing_zero_values_table(df):
        mis_val         = df.isnull().sum()
        mis_val_percent = round(df.isnull().mean().mul(100), 2)
        mz_table        = pd.concat([mis_val, mis_val_percent], axis=1)
        mz_table        = mz_table.rename(columns = {df.index.name:'col_name', 
                                                     0 : 'Valores ausentes', 
                                                     1 : '% de valores totais'})
        
        mz_table['Tipo de dados'] = df.dtypes
        mz_table                  = mz_table[mz_table.iloc[:,1] != 0 ]. \
                                     sort_values('% de valores totais', ascending=False)
        
        msg = "Seu dataframe selecionado tem {} colunas e {} " + \
              "linhas. \nExistem {} colunas com valores ausentes."
            
        print (msg.format(df.shape[1], df.shape[0], mz_table.shape[0]))
        
        return mz_table.reset_index()

In [ ]:
def describe(df):
    var = df.columns

    # Medidas de tendência central, média e mediana 
    ct1 = pd.DataFrame(df[var].apply(np.mean)).T
    ct2 = pd.DataFrame(df[var].apply(np.median)).T

    # Dispensão - str, min , max range skew, kurtosis
    d1 = pd.DataFrame(df[var].apply(np.std)).T
    d2 = pd.DataFrame(df[var].apply(min)).T
    d3 = pd.DataFrame(df[var].apply(max)).T
    d4 = pd.DataFrame(df[var].apply(lambda x: x.max() - x.min())).T
    d5 = pd.DataFrame(df[var].apply(lambda x: x.skew())).T
    d6 = pd.DataFrame(df[var].apply(lambda x: x.kurtosis())).T
    d7 = pd.DataFrame(df[var].apply(lambda x: (3 *( np.mean(x) - np.median(x)) / np.std(x) ))).T

    # concatenete 
    m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6, d7]).T.reset_index()
    m.columns = ['attrobutes', 'min', 'max', 'range', 'mean', 'median', 'std','skew', 'kurtosis','coef_as']
    
    return m

In [ ]:
def graf_bar_churn(df, col, title, xlabel, ylabel, tol = 0):
    
    #ax    = df.groupby(['churn_cat'])['churn_cat'].count()
    ax     = df    
    colors = cols
    
    if tol == 0: 
        total  = sum(ax)
        ax = (ax).plot(kind    ='bar',
                   stacked = True,
                    width = .5,
                   rot     = 0,
                   color   = colors)
    else:
        total  = tol     
        
        ax = (ax).plot(kind    ='bar',
                       stacked = True,
                       width = .5,
                       rot     = 0,figsize = (10,6),
                       color   = colors)

    #ax.yaxis.set_major_formatter(mtick.PercentFormatter())
    
    #y_fmt = tick.FormatStrFormatter('%.0f') 
    #ax.yaxis.set_major_formatter(y_fmt)

    title   = title + ' \n'
    xlabel  = '\n ' + xlabel 
    ylabel  = ylabel + ' \n'
    
    ax.set_title(title  , fontsize=22)
    ax.set_xlabel(xlabel, fontsize=13)
    ax.set_ylabel(ylabel, fontsize=13)    

    min = [0,23000000]
    #ax.set_ylim(min)
    
    for i in ax.patches:
        # get_width pulls left or right; get_y pushes up or down
        width, height = i.get_width(), i.get_height()
        x, y = i.get_xy()        
        
        ax.annotate(str(round((i.get_height() * 100.0 / total), 1) )+'%', 
                    (i.get_x()+.3*width, 
                     i.get_y()+.5*height),
                     color   = 'white',
                     weight = 'bold',
                     size   = 14)

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
def reduce_memory_usage(df, verbose=True):
    
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    
    for col in df.columns:
        
        col_type = df[col].dtypes
        
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
        
    return df

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    fig, ax = plt.subplots()
    ax.plot(fpr, tpr, "r-", label=label)
    ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="--", c=".3")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.rcParams['font.size'] = 12
    plt.title('ROC curve for TPS 09')
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Sensitivity)')
    plt.legend(loc="lower right")
    plt.grid(True)

In [ ]:
!mkdir Data
!mkdir Data/pkl
!mkdir Data/submission
!mkdir model
!mkdir model/preds
!mkdir model/optuna

!mkdir model/preds/test
!mkdir model/preds/test/n1
!mkdir model/preds/test/n2
!mkdir model/preds/test/n3

!mkdir model/preds/train
!mkdir model/preds/train/n1
!mkdir model/preds/train/n2
!mkdir model/preds/train/n3
!mkdir model/preds/param

## 1.3. Carregar Dados
Existem 2 conjuntos de dados que são usados na análise, eles são o conjunto de dados de treinamento e de teste. O principal uso do conjunto de dados de treino é treinar os modelos e usá-lo para prever o conjunto de dados de teste. 

Enquanto o arquivo de envio de amostra é usado para informar os participantes sobre a inscrição prevista para a competição. 

In [ ]:
path = '../input/tabular-playground-series-oct-2021/'
#path = 'Data/'

In [ ]:
%%time

df1_train  = pd.read_csv(path + 'train.csv', index_col='id')
df1_test   = pd.read_csv(path + 'test.csv',index_col='id')
df_submission = pd.read_csv(path + 'sample_submission.csv')

df1_train.shape, df1_test.shape, df_submission.shape

# <div class="alert alert-success">  1.0. EDA  </div> 

### 1.1.2. Dimensão do DataSet

In [ ]:
print('TREINO')
print('Number of Rows: {}'.format(df1_train.shape[0]))
print('Number of Columns: {}'.format(df1_train.shape[1]), end='\n\n')

print('TESTE')
print('Number of Rows: {}'.format(df1_test.shape[0]))
print('Number of Columns: {}'.format(df1_test.shape[1]))

### 1.1.3. Tipo de Dados

In [ ]:
df1_train.info()

In [ ]:
df1_test.info()

<div class="alert alert-info" role="alert"> 
 
`NOTA:` <BR>
    
- O dataset de treiro tem 2.1 GB com 1000000 de registros e 286 columas; 
- O dataset de teste tem 1.1 GB com 500000 de registros e 285 columas
    
Vamos fazer uma redução desses dataset, primeiro vamos identificar os tipos de dados que temos nos datasets.

</div>

In [ ]:
print(f'{3*"="} For Pandas {10*"="}\n{(df1_train.dtypes).value_counts()}')
print(f'\n{3*"="} For Datatable {7*"="}\n{(df1_test.dtypes).value_counts()}')

Temos dois tipos de dados nos datasets, sendo vamos criar duas variáveis para podermos entender o conteúdo das informações armazenadas. 

In [ ]:
feature_cat   = df1_test.select_dtypes(np.int64).columns.to_list()
feature_float = df1_test.select_dtypes(np.float64).columns.to_list()

df1_train[feature_cat].head()

In [ ]:
df1_train[feature_float].head()

<div class="alert alert-info" role="alert"> 
 
`NOTA:` <BR>
    
No primeiro dataset, temos variáveis categóricas que sofreram transformações para dammy e no segundo dataset temos variáveis quantitativas continuas, então agora vamos transformar os tipos dessas variávieis para fazermos a redução. 
    
</div>

In [ ]:
for col in feature_cat: 
    df1_train[col] = df1_train[col].astype(np.int8)
    df1_test[col] = df1_test[col].astype(np.int8)
    
for col in feature_float: 
    df1_train[col] = df1_train[col].astype(np.float32)
    df1_test[col] = df1_test[col].astype(np.float32)

In [ ]:
df1_train.info()

In [ ]:
df1_test.info()

Uma redução de 46.37% no dataset de treino e 43.91% no dataset de test. 

### 1.1.4. Idenficar Variáveis Ausentes (NA)
Vamos verificar os valores ausentes em cada variável conjunto de treinono e teste.

In [ ]:
missing = missing_zero_values_table(df1_train)
missing[:].style.background_gradient(cmap='Reds')

In [ ]:
missing = missing_zero_values_table(df1_test)
missing[:].style.background_gradient(cmap='Reds')

<div class="alert alert-info" role="alert"> 
    
`NOTA:` <br>

Não temos dados faltantes.
    
</div>

### 1.1.6. Estatística Descritiva
Abaixo estão as estatísticas básicas para cada variável que contém informações sobre contagem, média, desvio padrão, mínimo, 1º quartil, mediana, 3º quartil e máximo.

In [ ]:
df_num = df1_train.select_dtypes(np.number)
df_cat = df1_train.select_dtypes(exclude=[np.number])

df_num.shape, df_cat.shape

print('Temos {} variávies numéricas e {} categóricas.'.format(df_num.shape[1], df_cat.shape[1]))

#### 1.1.6.1. Atributos Numéricos

- Train

In [ ]:
df1_train[feature_float].describe().style.background_gradient(cmap='YlOrRd')

- Test

In [ ]:
df1_test[feature_float].describe().style.background_gradient(cmap='YlOrRd')

#### 1.1.6.1. Atributos Categóricos

In [ ]:
df1_train[feature_cat].columns

## 1.2. Análise Gráfica

### 1.2.1. Correlação
Vamos examinar a correlação entre as variáveis.

In [ ]:
df = df1_train.corr().round(5)

# Máscara para ocultar a parte superior direita do gráfico, pois é uma duplicata
mask = np.zeros_like(df)
mask[np.triu_indices_from(mask)] = True

# Making a plot
plt.figure(figsize=(16,16))
ax = sns.heatmap(df, annot=False, mask=mask, cmap="RdBu", annot_kws={"weight": "bold", "fontsize":13})

ax.set_title("Mapa de calor de correlação das variável", fontsize=17)

plt.setp(ax.get_xticklabels(), 
         rotation      = 90, 
         ha            = "right",
         rotation_mode = "anchor", 
         weight        = "normal")

plt.setp(ax.get_yticklabels(), 
         weight        = "normal",
         rotation_mode = "anchor", 
         rotation      = 0, 
         ha            = "right");

<div class="alert alert-info" role="alert"> 
 
`NOTA:` <br>
    
Como podemos ver, a correlação está entre ~-0,02 e 0, o que é muito pequeno. Portanto, as variáveis são fracamente correlacionados e negativas
    
</div>

### 1.2.2. Distribuição

#### 1.2.2.1. Train / Test

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

pie = ax.pie([len(df1_train), len(df1_test)],
             labels   = ["Train dataset", "Test dataset"],
             colors   = ["salmon", "teal"],
             textprops= {"fontsize": 15},
             autopct  = '%1.1f%%')

ax.axis("equal")
ax.set_title("Comparação de comprimento do conjunto de dados \n", fontsize=18)
fig.set_facecolor('white')
plt.show();

#### 1.2.2.2. Proporção das variáveis

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

plt.pie([len(feature_cat), len(feature_float)], 
        labels=['Categorical', 'Continuos'],
        textprops={'fontsize': 13},
        autopct='%1.1f%%')

#ax.axis("equal")
ax.set_title("Comparação variáveis continuas/categóricas \n Dataset Treino/Teste", fontsize=18)
fig.set_facecolor('white')
plt.show()

#### 1.2.2.1. Target
Vamos ver as ocorrências de números individuais do conjunto de dados de treino.

In [ ]:
L    = len(df1_train.columns[0:60])
nrow = int(np.ceil(L/6))
ncol = 6

remove_last= (nrow * ncol) - L

fig, ax = plt.subplots(nrow, ncol,figsize=(24, 30))
fig.subplots_adjust(top=0.95)
i = 1

for feature in df1_train.columns[0:60]:
    
    plt.subplot(nrow, ncol, i)
    
    ax = sns.kdeplot(df1_train[feature], shade=True, color='salmon',  alpha=0.5, label='train')
    ax = sns.kdeplot(df1_test[feature], shade=True, color='teal',  alpha=0.5, label='test')
    plt.xlabel(feature, fontsize=9)
    plt.legend()
    
    i += 1
    
plt.suptitle('DistPlot: train & test data', fontsize=20)
plt.show()

<div class="alert alert-info" role="alert"> 
 
`NOTA:` <br>

- Os conjuntos de treinamento e teste têm aproximadamente as mesmas distribuições em termos de variáveis; <br>
- Temos poucas variáveis com distribuições normais; <br>
- A maioria das variáveis tem distribuições distorcidas. <br>

Precisamos pensar em como fazer tudo isso normalmente distribuído se decidirmos usar modelos não baseados em árvore. <br>

> A verificação da correlação não revelou relações significativas entre as características (a maioria estava entre -0.02  e 0).

</div>

#### 1.2.2.1. Detecção de Outlier

##### 1.2.2.1.1. Data Train 

In [ ]:
df_plot = ((df1_train - df1_train.min())/(df1_train.max() - df1_train.min()))
fig, ax = plt.subplots(4, 1, figsize = (25,25))

for i, (x) in enumerate([(1,30), (30,60), (60,90), (90,120)]): 
    sns.boxplot(data = df_plot.iloc[:, x[0]:x[1] ], ax = ax[i]);

##### 1.2.2.1.2. Data Test

In [ ]:
df_plot = ((df1_test - df1_test.min())/(df1_test.max() - df1_test.min()))
fig, ax = plt.subplots(4, 1, figsize = (25,25))

for i, (x) in enumerate([(1,30), (30,60), (60,90), (90,120)]): 
    sns.boxplot(data = df_plot.iloc[:, x[0]:x[1] ], ax = ax[i]);

<div class="alert alert-info" role="alert"> 
 
`NOTA:` <br>
    
Acima observamos que temos muitos outliers em ambos conjunto de dados, no processamento vamos trartá-los.

</div>

#### 1.2.2.3. Target
A variável alvo tem os valores 0 e 1, vamos verificar a distribuição da target.

In [ ]:
plt.figure(figsize=(8, 6))
ax = sns.countplot(x=df1_train['target'], palette='viridis')
ax.set_title('Distribuição da variável Target', fontsize=20, y=1.05)

sns.despine(right=True)
sns.despine(offset=10, trim=True)

<div class="alert alert-info" role="alert"> 
 
`NOTA:` <br>
    
Podemos observar no gráfico acima que não temos desbalanceamento nos dados. 
    
</div>

#### 1.2.2.4. Variáveis preditoras  vs Target.

In [ ]:
L    = len(df1_train.columns[0:60])
nrow = int(np.ceil(L/6))
ncol = 6
i    = 1
remove_last= (nrow * ncol) - L

fig, ax = plt.subplots(nrow, ncol,figsize=(24, 30))
fig.subplots_adjust(top=0.95)

for feature in df1_train.columns[0:60]:
    
    plt.subplot(nrow, ncol, i)
    
    ax = sns.kdeplot(df1_train[feature], 
                     shade    = True, 
                     palette  = 'viridis',  
                     alpha    = 0.5, 
                     hue      = df1_train['target'], 
                     multiple = "stack")
    
    plt.xlabel(feature, fontsize=9)
        
    i += 1
    
    gc.collect()
    
plt.suptitle('DistPlot: Variável de treino vs target', fontsize=20)
plt.show()

# <div class="alert alert-success">  2. Split Train/Test </div> <br>

In [ ]:
gc.collect()

X      = df1_train.drop('target', axis=1)
y      = df1_train['target']
X_test = df1_test
cols   = X.columns


X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size    = 0.2,
                                                      shuffle      = True, 
                                                      stratify     = y,
                                                      random_state = 0)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape , X_test.shape

Nesta etapa, treinaremos nosso modelo **XGBClassifier** de linha de base simples, em relação ao tratamento dos dados, vamos fazer apenas o scaler nesta etapa.

In [ ]:
X_test.head()

# <div class="alert alert-success">  2. Modelo Baseline XGB </div> <br>

In [ ]:
%%time 
seed   = 12359
params = {'random_state'  : seed,          
          'predictor'     : 'gpu_predictor',
          'tree_method'   : 'gpu_hist',
          'eval_metric'   : 'auc'}

model_baseline = xgb.XGBClassifier(**params)

scalers = [None, StandardScaler(), RobustScaler(), MinMaxScaler(), 
           MaxAbsScaler(), QuantileTransformer(output_distribution='normal', random_state=0)]

for scaler in scalers: 
    
    if scaler!=None:
        X_train_s = scaler.fit_transform(X_train)
        X_valid_s = scaler.fit_transform(X_valid)
    else:
        X_train_s = X_train
        X_valid_s = X_valid
                
    model_baseline.fit(X_train_s, y_train, verbose = False)
    y_hat = model_baseline.predict_proba(X_valid_s)[:, 1]
    
    fpr, tpr, thresholds = metrics.roc_curve(y_valid, y_hat)
    
    auc = metrics.auc(fpr, tpr)
    print('Validaçao AUC: {:2.5f} => {}'.format(auc, scaler))
    
print()

<div class="alert alert-info" role="alert"> 
 
`NOTA:` <br>
    
Com scaler RobustScaler obtivemos uma AUC de 0.85144, como estamos fazer apenas uma validação (treino/validação), neste caso a pontuação do score pode ser afetada por aleatoriedade dos dados, sendo assim, vamos fazer uma validação cruzada para termos uma estimativa robusta.  <br>

Para o treinamento do modelo foi criado a função abaixo que tem a finalidade de treinar um conjunto de scalers para um determinado modelo, durante o treinamento será exibido os resultados e no final será retornado o melhor modelo com as variáveis e seus scores de importância.
    
</div>

In [ ]:
path='Data/'

def cross_val_model(model, scalers, name_file_submission, FOLDS=5, verbose=False, seed=59): 
    
    mdl_train   = []
    feature_imp = 0 
    
    for scaler in scalers: 

        df_submission.claim = 0
        auc_best            = 0   
        feature_imp_best    = 0       
        auc                 = []
        lloss               = []
        f1                  = []
        kfold               = KFold(n_splits=FOLDS, random_state=seed, shuffle=True)

        if scaler!=None:
            X_ts = scaler.fit_transform(X_test.copy())
        else:
            X_ts = X_test.copy()

        print('='*80)
        print('Scaler: {}'.format(scaler))
        print('='*80)

        for i, (train_idx, test_idx) in enumerate(kfold.split(X_train)):

            i+=1

            X_tr, y_tr = X_train.iloc[train_idx], y_train.iloc[train_idx]
            X_vl, y_vl = X_train.iloc[test_idx], y_train.iloc[test_idx]

            # Scaler
            if scaler!=None:    
                X_tr = scaler.fit_transform(X_tr)
                X_vl = scaler.fit_transform(X_vl)                

            eval_set     = [(X_tr,y_tr), (X_vl,y_vl)]            
            n_estimators = model.get_params()['n_estimators'] 

            if n_estimators==100: 
                early_stopping_rounds = 20
            else: 
                early_stopping_rounds = 200

            model = model.fit(X_tr, y_tr, 
                              eval_set              = eval_set,
                              early_stopping_rounds = early_stopping_rounds, 
                              verbose               = verbose,
                             )
            
            best_ntree = model.best_ntree_limit
            
            y_hat_prob = model.predict_proba(X_vl, ntree_limit=best_ntree)[:, 1] #   
            y_hat      = (y_hat_prob >.5).astype(int) 

            fpr, tpr, thresholds = metrics.roc_curve(y_vl, y_hat_prob)

            log_loss_     = metrics.log_loss(y_vl, y_hat_prob)                
            f1_score_     = metrics.f1_score(y_vl, y_hat)        
            auc_          = metrics.auc(fpr, tpr)    

            stop = ''
            
            if n_estimators > best_ntree: 
                stop = '*'
                
            msg = '[Fold {}] AUC: {:.5f} - F1: {:.5f} - L. LOSS: {:.5f} {}'
            print(msg.format(i, auc_, f1_score_,log_loss_, stop))

            # Getting mean feature importances (i.e. devided by number of splits)
            feature_imp  += model.feature_importances_ / FOLDS
            
            df_submission['target'] += model.predict_proba(X_ts)[:, 1] / FOLDS

            f1.append(f1_score_)
            lloss.append(log_loss_)
            auc.append(auc_)
                        
        auc_mean   = np.mean(auc)
        auc_std    = np.std(auc)
        lloss_mean = np.mean(lloss)
        f1_mean    = np.mean(f1)
        
        if auc_mean > auc_best: 
            auc_best          = auc_mean
            f1_best           = f1_mean
            lloss_best        = lloss_mean
            model_best        = model
            feature_imp_best  = feature_imp
            scaler_best       = scaler
                                    
        print('-'*80)
        msg = '[Mean Fold] AUC: {:.5f}(Std:{:.5f}) - F1: {:.5f} - L. LOSS: {:.5f}'
        print(msg.format(auc_mean,auc_std, f1_mean, lloss_mean))
        print('='*80)
        print('')

        # Gerar o arquivo de submissão 
        name_file_sub = 'submission/' + name_file_submission + '_' + str(scaler).lower()[:4] + '.csv'
        df_submission.to_csv(path + name_file_sub, index = False)

        gc.collect()
        
    print()
    print('='*80)
    print('Scaler Best: {}'.format(scaler_best))
    print('AUC        : {:2.5f}'.format(auc_best))
    print('F1-Score   : {:2.5f}'.format(f1_best))
    print('L. Loss    : {:2.5f}'.format(lloss_best))
    print('='*80)
    print()
            
    gc.collect()  
    
    return model_best, feature_imp_best 

In [ ]:
%%time

gc.collect()

seed    = 12359
scalers = [None, StandardScaler(), RobustScaler(), MinMaxScaler(), 
           MaxAbsScaler(), QuantileTransformer(output_distribution='normal', random_state=0)]

model_best, feat_imp_best = cross_val_model(xgb.XGBClassifier(**params), 
                                            scalers, 
                                            '001_xgb_baseline', 
                                            FOLDS = 5, 
                                            seed  = seed
                                            )

<div class="alert alert-info" role="alert">

Na validação cruzada obtivemos uma `AUC` de `0.85093` com um desvio padrão de `0.00088` sem fazer o scaler, vamos gerar dois arquivos com as melhores pontuação de AUC para submissão. <br>
    
`NOTA:` <br>
   
Na validação cruzada que foi realizada, mostra que não fazer a normalização temos a melhor AUC, e isso é confirmado na submissão da competição, abaixo as duas submissões:   <br>
    
- AUC: 0.85383 => Sem normalização; <br> 
- AUC: 0.85246 => RobustScaler <br>
    
Vamos treinar três modelos, sendo que vamos acrescentar o parametros `n_estimators` que indica o número de arvores para o treinamento do modelo, próximos notebooks vamos fazer ajuste em diversos parametros para ajusdar no score.
    
</div>

In [ ]:
%%time
    
seed   = 112359    
params = {'random_state'  : seed,         
          'n_estimators'  : 1000,
          'predictor'     : 'gpu_predictor',
          'tree_method'   : 'gpu_hist',
          'eval_metric'   : 'auc'}

scalers = [None, 
           RobustScaler(), 
           QuantileTransformer(output_distribution='normal', random_state=0)]

model_best, feat_imp_best  = cross_val_model(xgb.XGBClassifier(**params), 
                                             scalers, 
                                             '002_xgb_bl_n_estimators_1000', 
                                             FOLDS = 5, 
                                             seed  = seed
                                            )

<div class="alert alert-info" role="alert"> 
 
`NOTA:` <br>
Na submissão na competição obtivemos uma AUC de `0.85407` com o QuantileTransformer em relação ao treinamento do modelo, vamos fazer uma pequena análise do comportamento do modelo na próxima seção.
   
</div>

## 2.1. Análise do Modelo 


In [ ]:
model_best

### 2.1.1. Número de Estimadores

In [ ]:
results     = model_best.evals_result()
ntree_limit = model_best.best_ntree_limit

plt.figure(figsize=(7,5))
plt.plot(results["validation_0"]["auc"], label="Treinamento")
plt.plot(results["validation_1"]["auc"], label="Validação")


plt.axvline(ntree_limit, 
            color="gray", 
            label="N. de árvore ideal {}".format(ntree_limit))

plt.xlabel("Número de árvores")
plt.ylabel("AUC")
plt.legend();

<div class="alert alert-info" role="alert"> 
    
`NOTA:` <br>
Acima recuperamos as informações de treinamento do nosso modelo, podemos observar que o número de 1000 estimadores é mais que suficiente para o treinamento do modelo, o ideal é que fique em torno de 75 à 95 para esses dados e com a utilização dos parametros padrões que devem ser ajustados para o `XGB`. <br>
    
    
Vamos agora utilizar o modelo treinado que foi retornado pela função e vamos fazer a previsão para novos dados que o modelo não viu no treinamento, para termos uma ideia da generalização do modelo, lembrando que o modelo que foi treinado utiliza 1000 estimadores (arvores), sendo assim, vamos utilizar na previssão 86 estimadores utilizando o parametro `ntree_limit` ao fazermos as previsões.

</div>

In [ ]:
%%time
threshold =.5

y_pred_prob = model_best.predict_proba(X_valid, ntree_limit=ntree_limit)[:, 1] 
y_pred      = (y_pred_prob > threshold).astype(int)

f1_    = metrics.f1_score(y_valid, y_pred)
auc_   = metrics.roc_auc_score(y_valid, y_pred)
lloss_ = metrics.log_loss(y_valid, y_pred_prob) 
    
print('AUC     : {:2.5f}'.format(auc_))
print('AUC     : {:2.5f}'.format(auc))
print('F1-Score: {:2.5f}'.format(f1_))
print('L. Loss : {:2.5f}'.format(lloss_))
print()

<div class="alert alert-info" role="alert"> 
    
`NOTA:` <br>
    
Observando o resultado acima nos dados de validação, o nosso modelo parece que está com um pequeno overfitting em relação ao treinamento, mesmo assim é um modelo bom, pois está generalizando pelo resultado na submissão na competição.      
    
</div>

### 2.1.2. Matriz de Confusão

In [ ]:
metrics.plot_confusion_matrix(model_best, X_valid, y_valid, cmap='inferno')
plt.title('Confusion matrix')
plt.grid(False)
plt.show()

In [ ]:
threshold    = .5
y_pred_valid = (y_pred_prob > threshold).astype(int)
f1_          = metrics.f1_score (y_valid, y_pred_valid)
auc_         = metrics.roc_auc_score(y_valid, y_pred_prob)

print(metrics.classification_report(y_valid, y_pred_valid))
print('')
print('AUC     : {:2.5f}'.format(auc_))
print('F1-score: {:2.5f}'.format(f1_))


<div class="alert alert-info" role="alert"> 
    
**`NOTA:`** <br>
   
    
</div>

### 2.1.3 Curva ROC

In [ ]:
fpr, tpr, thresholds = metrics. roc_curve(y_valid, y_pred)

plot_roc_curve(fpr, tpr, label="XGB")
plt.show()

<div class="alert alert-info" role="alert"> 
    
`NOTA:` <br>
   
    
</div>

### 2.2.4. Feature Importances  

In [ ]:
feature_imp_ = feat_imp_best

In [ ]:
df               = pd.DataFrame()
df["Feature"]    = X.columns
df["Importance"] = feature_imp_ / feature_imp_.sum()

df.sort_values("Importance", axis=0, ascending=False, inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(13, 70))
bars    = ax.barh(df["Feature"], 
                  df["Importance"], 
                  height    = 0.4,
                  color     = "mediumorchid", 
                  edgecolor = "black")

ax.set_title("Feature importances", fontsize=30, pad=15)
ax.set_ylabel("Feature name", fontsize=20, labelpad=15)
#ax.set_xlabel("Feature importance", fontsize=20, labelpad=15)
ax.set_yticks(df["Feature"])
ax.set_yticklabels(df["Feature"], fontsize=13)
ax.tick_params(axis="x", labelsize=15)
ax.grid(axis="x")

# Adicionando rótulos na parte superior
ax2 = ax.secondary_xaxis('top')
#ax2.set_xlabel("Feature importance", fontsize=20, labelpad=13)
ax2.tick_params(axis="x", labelsize=15)
ax.margins(0.05, 0.01)

# Inverter a direção do eixo y 
plt.gca().invert_yaxis()